In [1]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
print "Libraries loaded"

Libraries loaded


Now that we have our dataset properly ready, we can start building a simple deep neural net for further evaluation. Let's begin by loading the dataset.

In [2]:
pickle_file = 'dataset.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (70000, 32, 32, 3), (70000,))
('Validation set', (10000, 32, 32, 3), (10000,))
('Test set', (20000, 32, 32, 3), (20000,))


Now let's Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 32
num_labels = 10
depth = 3

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size * depth)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (70000, 3072), (70000, 10))
('Validation set', (10000, 3072), (10000, 10))
('Test set', (20000, 3072), (20000, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default(): # describe the computation graph to ran

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size * depth, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [10]:
num_steps = 801

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Start processing.
Initialized
Loss at step 0: 25.204840
Training accuracy: 10.7%
Validation accuracy: 10.0%
Loss at step 100: 13.802734
Training accuracy: 10.9%
Validation accuracy: 10.2%
Loss at step 200: 17.523998
Training accuracy: 11.2%
Validation accuracy: 10.7%
Loss at step 300: 12.195518
Training accuracy: 12.2%
Validation accuracy: 10.5%
Loss at step 400: 22.071648
Training accuracy: 12.7%
Validation accuracy: 11.0%
Loss at step 500: 19.041563
Training accuracy: 10.9%
Validation accuracy: 10.5%
Loss at step 600: 24.776970
Training accuracy: 10.9%
Validation accuracy: 10.5%
Loss at step 700: 23.784254
Training accuracy: 13.7%
Validation accuracy: 11.3%
Loss at step 800: 13.772348
Training accuracy: 13.2%
Validation accuracy: 10.3%
Test accuracy: 10.8%


By looking at the small values of training and testing accuracy, we can say that this model seems to suffer from high variance, that is, the model is too simple to capture the underlying structure in the dataset.  A possible solution is to go deeper and explore much more complex models that might be able to learn the structure of the dataset.

Let's swtich to Stochastic Gradient Descent.

In [18]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size * depth))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size * depth, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
        
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 25.714630
Minibatch accuracy: 11.7%
Validation accuracy: 9.8%
Minibatch loss at step 500: 24.778267
Minibatch accuracy: 13.3%
Validation accuracy: 10.8%
Minibatch loss at step 1000: 28.478218
Minibatch accuracy: 9.4%
Validation accuracy: 10.5%
Minibatch loss at step 1500: 14.777555
Minibatch accuracy: 10.2%
Validation accuracy: 10.6%
Minibatch loss at step 2000: 29.448759
Minibatch accuracy: 12.5%
Validation accuracy: 11.2%
Minibatch loss at step 2500: 21.852587
Minibatch accuracy: 16.4%
Validation accuracy: 11.5%
Minibatch loss at step 3000: 22.580236
Minibatch accuracy: 12.5%
Validation accuracy: 11.0%
Test accuracy: 10.8%


Let's transform the Multinomial Logistic Regretion with SGD example into a 1-hidden neural network with Rectified Liner Units (RELU)

In [23]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size*image_size*depth))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  hidden_layer_size = 4096

  # Variables.
  # defines the weights and biases for the hidden layer's input
  weights1 = tf.Variable(
    tf.truncated_normal([image_size*image_size*depth, hidden_layer_size]))
    
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))

  # Training computation.
  hidden_layer_in = tf.matmul(tf_train_dataset, weights1) + biases1

  # apply the rectified liner regretion 
  hidden_layer_in = tf.nn.relu(hidden_layer_in)
  
  # defines the weights and biases for the hidden layer's output
  weights2 = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    
  biases2 = tf.Variable(tf.zeros([num_labels]))

  hidden_layer_out = tf.matmul(hidden_layer_in, weights2) + biases2

  # apply cross entropy
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(hidden_layer_out, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(hidden_layer_out)
  
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights2) + biases2)
  
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights2) + biases2)

In [24]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1508.976929
Minibatch accuracy: 9.4%
Validation accuracy: 9.8%
Minibatch loss at step 500: 433.308105
Minibatch accuracy: 18.0%
Validation accuracy: 22.6%
Minibatch loss at step 1000: 254.485489
Minibatch accuracy: 24.2%
Validation accuracy: 22.4%
Minibatch loss at step 1500: 179.885910
Minibatch accuracy: 21.1%
Validation accuracy: 18.7%
Minibatch loss at step 2000: 117.050255
Minibatch accuracy: 27.3%
Validation accuracy: 24.3%
Minibatch loss at step 2500: 81.481445
Minibatch accuracy: 32.8%
Validation accuracy: 24.0%
Minibatch loss at step 3000: 124.745567
Minibatch accuracy: 28.9%
Validation accuracy: 25.2%
Test accuracy: 25.7%


In [9]:
batch_size = 128

# define the input variable
# the input variable will receive the image's pixels for every batch
tf_train_dataset = tf.placeholder(tf.float32, [None, image_size * image_size * depth]) # [128 x 784] matrix 
tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

# placeholder to determine the percentage of data that will be dropped out
keep_prob = tf.placeholder(tf.float32)

# load the valid and test datasets
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)

##############################################################
## Layer 1
##############################################################
# define the hidden layer size
hidden_layer_size = 2048

# define the weights (parameters) of the first layer
W_layer1 = tf.Variable(tf.truncated_normal([image_size*image_size*depth, hidden_layer_size], stddev=0.03)) # [784x1024] Matrix 

# define the biases for the firtst layer
b_layer1 = tf.Variable(tf.zeros([hidden_layer_size]))

# training computation
hidden_layer = tf.matmul(tf_train_dataset, W_layer1) + b_layer1 # [128 x 1024] Matrix

# apply the relu (Rectified Linear Regression) function 
relu_hidden_layer = tf.nn.dropout(tf.nn.relu(hidden_layer), keep_prob)

##############################################################
## Layer 2
##############################################################
# define the hidden layer size
hidden_layer2_size = 1024

# define the weights (parameters) of the first layer
W_layer2 = tf.Variable(tf.truncated_normal([hidden_layer_size, hidden_layer2_size], stddev=0.03)) # [1024x2048] Matrix 

# define the biases for the firtst layer
b_layer2 = tf.Variable(tf.zeros([hidden_layer2_size])) # [2048]

# training computation
hidden_layer2 = tf.matmul(relu_hidden_layer, W_layer2) + b_layer2 # [128 x 2048] Matrix

# apply the relu (Rectified Linear Regression) function 
relu_hidden_layer2 = tf.nn.dropout(tf.nn.relu(hidden_layer2), keep_prob)

##############################################################
## Layer 3
##############################################################
# define the hidden layer size
# hidden_layer3_size = 1024

# define the weights (parameters) of the first layer
# W_layer3 = tf.Variable(tf.truncated_normal([hidden_layer2_size, hidden_layer3_size], stddev=0.03)) # [2048x2048] Matrix 

# define the biases for the firtst layer
# b_layer3 = tf.Variable(tf.zeros([hidden_layer3_size])) # [2048]

# training computation
# hidden_layer3 = tf.matmul(relu_hidden_layer2, W_layer3) + b_layer3 # [128 x 2048] Matrix

# apply the relu (Rectified Linear Regression) function 
# relu_hidden_layer3 = tf.nn.dropout(tf.nn.relu(hidden_layer3), keep_prob)

##############################################################
## Layer 4
##############################################################

# define the parameters for the second layer
W_layer3 = tf.Variable(tf.truncated_normal([hidden_layer2_size, num_labels], stddev=0.03)) # [1024 x 10] Matrix

# define the biases for the second layer 
b_layer3 = tf.Variable(tf.zeros([num_labels]))

# training computation
logits = tf.matmul(relu_hidden_layer2, W_layer3) + b_layer3 # [128 x 10]

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

# apply regularization
regularizers = tf.nn.l2_loss(W_layer1) + tf.nn.l2_loss(W_layer2) + tf.nn.l2_loss(W_layer3)
loss += 4e-4 * regularizers 

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.5
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           decay_steps = batch_size, 
                                           decay_rate = 0.95, 
                                           staircase=True)

# optimize the loss function using gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

train_predictions = tf.nn.softmax(logits)

# run the validation dataset using the trained netword (weights and biases)
valid_prediction_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, W_layer1) + b_layer1)
valid_prediction_hidden_layer = tf.nn.relu(tf.matmul(valid_prediction_hidden_layer, W_layer2) + b_layer2)
#valid_prediction_hidden_layer = tf.nn.relu(tf.matmul(valid_prediction_hidden_layer, W_layer3) + b_layer3)
valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction_hidden_layer, W_layer3) + b_layer3)

# run the test dataset in the trained netword (weights and biases)
test_prediction_hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_test_dataset, W_layer1) + b_layer1), 1.0)
test_prediction_hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(test_prediction_hidden_layer, W_layer2) + b_layer2), 1.0)
#test_prediction_hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(test_prediction_hidden_layer, W_layer3) + b_layer3), 1.0)
test_prediction = tf.nn.softmax(tf.matmul(test_prediction_hidden_layer, W_layer3) + b_layer3)

In [ ]:
# properly initialize the tensorflow variables
init = tf.initialize_all_variables()

# initialize the model in run the operation to initialize the variables
sess = tf.Session()
sess.run(init)

num_steps = 3001
learning_rate_decay = []

for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    
    opt, l, l_rate, predictions = sess.run(
      [optimizer, loss, learning_rate, train_predictions], feed_dict=feed_dict)
    
    
    learning_rate_decay.append(l_rate)
    
    if (step % 500 == 0):
        print("Offset:", offset)
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy %.1f%%" % accuracy(valid_prediction.eval(session=sess), valid_labels))
        
print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(session=sess), test_labels))

import matplotlib.pyplot as plt

# display a learning rate decreasing in a graph
plt.plot(learning_rate_decay)
plt.grid(1)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()